# Candidates of keyword

## Load model

In [2]:
import json
import os
from training import create_PL
from modules import get_pl_keywords
import requests

## Load data

In [24]:
def get_all_raw_pl():
     
    print("load data")
    total_data = [] 
    r = requests.get('https://steevebot.ml/all')  
    ori_data = r.json()
    key = list(ori_data.keys())
    
    for k in key: 
        data = []
    
        for num, job_num in enumerate(ori_data[k]):
            if num%500 ==0:
                print(num)

            pl_des = get_pl_keywords(job_num["jobDescription"])
            pl_ski = get_pl_keywords(job_num["skills"])
            data.append(pl_des+pl_ski)
#             print(data)
            str_data = ''
            str_data = " ".join(pl_des+pl_ski) 
        
            # get location
            
            ##### save PL and location to DB ####
#             create_PL(job_num["JobID"],k,str_data,location)

        total_data.append(data)
    return total_data,str_data

In [7]:
r = requests.get('https://steevebot.ml/all')
ori_data = r.json()
key = list(ori_data.keys())

In [25]:
a,b = get_all_raw_pl()

load data
0
500
1000
1500
2000
2500
3000
3500
0
500
1000
1500
2000
2500
3000
3500


In [13]:
for k in key:
    print(k)
print(key[0])

Frontend
Backend
Frontend


In [29]:
print(type(b))

<class 'str'>


In [4]:
data = get_all_raw_pl()

load data
0
500
1000
1500
2000
2500
3000
3500
0
500
1000
1500
2000
2500
3000
3500


In [9]:
def training_DNN:
    from DNN_data import get_Dnn_model
    total_data = get_all_raw_pl()
    
    # training DNN model
    get_Dnn_model(total_data)
    
def get_jobs(cv,pl_cnt):
    from DNN_data import get_predict_field
    from modules import pick_k_jobs
    
    try:
        pl_cnt = TFIDF.TFIDF.tfidf_dict
        
        print('pl_cnt exists')
    else:
        total_data = get_all_raw_pl()
        pl_cnt, word = TFIDF.TFIDF.get_tfidf(total_data)
        
    
    cv_PL = get_pl_keywords(cv, pl_cnt)
    predict_field = get_predict_field(cv_PL)
    
    #### get field data from DB ####
    p = get_field_PL(predict_field)
    # for j in p:
        posts_predict_field.append(j.Job.job_ID,j.Job.Field,j.PL,j.PL_location)
    # posts_predict_field = GET DB DATA
    
    job_candidates = pick_k_jobs(cv_PL, posts_predict_field)
    
def get_applicants():
    

load data
0
500
1000
1500
2000
2500
3000
3500
0
500
1000


KeyboardInterrupt: 

In [6]:
for i in data[0]:
    print(i)
# print(data[0])

['security', 'security', 'Dell', 'build', 'deployment', 'redux', 'React', 'frontend', 'open-source', 'build', 'webpack', 'frameworks', 'javascript', 'web', 'frameworks', 'Java', 'css3', 'html5', 'security', 'security', 'React', 'redux']
['reactjs', 'build', 'interface', 'module', 'validation', 'this', 'this', 'this', 'html5', 'css3', 'BEM', 'reactjs', 'redux', 'redux', 'javascript', 'jasmine', 'mocha', 'system', 'Java', 'jsp', 'servlets', 'eclipse', 'jira', 'Nodejs', 'npm', 'apache', 'web', 'UNIX', 'hyperlink', 'http', 'orientation']
['.net', 'web', 'asp.net', 'jQuery', 'html', 'javascript', 'css', 'c#', 'sql', 'database', 'web', 'c#', '.net', 'web', 'sql', 'database', 'sql', 'sql', 'system', 'database', 'networking', 'c#', 'asp.net', 'sql', 'html', 'css']
['build', 'this', 'wordpress', 'wordpress', 'wordpress', 'plugins', 'build', 'web', 'website', 'website', 'PHP', 'javascript', 'html', 'css', 'wordpress', 'security', 'less', 'rss', 'xml', 'wordpress', 'wordpress', 'wordpress', 'word

['Java', 'Java', 'build', 'tdd', 'web', 'spring', 'spring', 'hibernate', 'process', 'html', 'css', 'javascript', 'javascript', 'frameworks', 'Java', 'sass', 'React', 'webpack', 'rest', 'Nodejs', 'Java', 'spring', 'deployment', 'docker', 'vagrant', 'junit', 'jasmine', 'Jenkins', 'chef', 'git', 'performance', 'linux', 'UNIX', 'Java', 'spring', 'hibernate', 'html', 'css', 'javascript']
['mobile', 'build', 'api', 'mobile', 'mobile', 'performance', 'mobile', 'mobile', 'angularjs', 'Node.js', 'mobile', 'ios', 'android', 'mobile', 'ios', 'angular']
['this', 'Node.js', 'Node.js', 'security', 'nosql', 'lambda', 'azure', 'rest', 'this', 'Node.js', 'Node.js', 'frameworks', 'jasmine', 'mocha', 'cucumber', 'nosql', 'redis', 'module', 'webpack', 'gulp', 'npm', 'javascript', 'typescript', 'coffeescript', 'Jenkins', 'vsts', 'process', 'authorization', 'security', 'validation', 'web', 'angular', 'React', 'api', 'api', 'azure', 'api', 'azure', 'Nodejs', 'rest', 'jasmine', 'mocha', 'typescript']
['Java',

In [4]:
tf_idf = TFIDF(total_pl)
# from TFIDF import tf_idf
pl_cnt, words = tf_idf.get_tfidf()
print(pl_cnt)

NameError: name 'total_pl' is not defined

In [ ]:
#### Return jobID, Field,PL[]